In [15]:
quiet_library <- function(...) { suppressPackageStartupMessages(library(...)) }
quiet_library(purrr)
quiet_library(dplyr)
quiet_library(matrixStats)

In [2]:
res_files <- list.files("model_cors/", full.names = TRUE)

In [3]:
res_list <- map(res_files, read.csv)

In [4]:
head(res_list[[1]])

,GeneScoreMatrix_name,GeneExpressionMatrix_name,cor,padj,pval,GeneScoreMatrix_seqnames,GeneScoreMatrix_start,GeneScoreMatrix_end,GeneScoreMatrix_strand,GeneScoreMatrix_idx,GeneScoreMatrix_matchName,GeneExpressionMatrix_seqnames,GeneExpressionMatrix_idx,GeneExpressionMatrix_start,GeneExpressionMatrix_end,GeneExpressionMatrix_strand,GeneExpressionMatrix_matchName
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<int>,<int>,<int>,<chr>,<chr>,<int>,<int>,<int>,<int>,<chr>
1,OR4F5,OR4F5,NA,NA,NA,chr1,69091,70008,1,1,OR4F5,chr1,3,65418,69055,3,OR4F5
2,FAM87B,FAM87B,0.054700945,1,0.2220848,chr1,817371,819837,1,4,FAM87B,chr1,16,817370,817371,3,FAM87B
3,LINC01128,LINC01128,-0.038221196,1,0.3937566,chr1,825138,859446,1,5,LINC01128,chr1,18,827597,860227,3,LINC01128
4,LINC00115,LINC00115,0.034887385,1,0.4363391,chr1,827522,826206,2,6,LINC00115,chr1,17,827521,827522,3,LINC00115
5,FAM41C,FAM41C,-0.005026375,1,0.9107342,chr1,876903,868071,2,7,FAM41C,chr1,19,870200,876903,3,FAM41C
6,LINC02593,LINC02593,NA,NA,NA,chr1,919692,916870,2,8,LINC02593,chr1,23,919691,919692,3,LINC02593


In [5]:
res_df <- map_dfc(res_list, "cor")

New names:
* NA -> ...1
* NA -> ...2
* NA -> ...3
* NA -> ...4
* NA -> ...5
* ...



In [6]:
names(res_df) <- sub(".csv","",basename(res_files))

In [8]:
res_mat <- as.matrix(res_df)
rownames(res_mat) <- res_list[[1]]$GeneExpressionMatrix_name

In [10]:
res_mat[is.na(res_mat)] <- 0

### For each gene, which model works best/worst

In [36]:
cor_df <- data.frame(gene = rownames(res_mat),
                     max_cor = rowMaxs(res_mat),
                     min_cor = rowMins(res_mat),
                     mean_cor = rowMeans(res_mat),
                     median_cor = rowMedians(res_mat),
                     diff_cor = rowMaxs(res_mat) - rowMins(res_mat),
                     max_model = apply(res_mat, 1, function(x) { colnames(res_mat)[which.max(x)] }),
                     min_model = apply(res_mat, 1, function(x) { colnames(res_mat)[which.min(x)] }))

Genes for which model selection matters the most:

In [37]:
cor_df %>%
  arrange(desc(diff_cor)) %>%
  head(20)

,gene,max_cor,min_cor,mean_cor,median_cor,diff_cor,max_model,min_model
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
1,TBC1D5,0.8690188,-0.5803292,0.1295342,-0.51914398,1.449348,GeneModel-GB-NoBoundary-Exponential-3_cor,GeneBody_0_0_cor
2,ETS1,0.8155334,-0.3698066,0.2945703,0.32980829,1.185340,GeneModel-GB-NoBoundary-Exponential-1_cor,GeneModel-TSS-Constant-2_cor
3,BANK1,0.9413987,-0.2196137,0.4627869,0.09577477,1.161012,GeneModel-GB-Exponential-Extend-18_cor,GeneModel-TSS-Constant-1_cor
4,PACS1,0.7900915,-0.3567469,0.2734576,0.21005930,1.146838,GeneModel-GB-NoBoundary-Exponential-3_cor,GeneModel-TSS-Constant-1_cor
5,LINC00612,0.7772353,-0.3310071,0.4543011,0.54225510,1.108242,GeneBody_0_0_cor,GeneModel-TSS-Constant-1_cor
6,A2M-AS1,0.7824948,-0.3182763,0.1924655,-0.10055424,1.100771,GeneModel-GB-NoBoundary-Exponential-4_cor,GeneModel-TSS-Constant-1_cor
7,AKNA,0.8112003,-0.2843566,0.7517624,0.77023468,1.095557,GeneModel-GB-Exponential-1_cor,GeneModel-TSS-Constant-1_cor
8,A2M,0.7799945,-0.3036986,0.1995983,-0.09277289,1.083693,GeneModel-GB-NoBoundary-Exponential-4_cor,GeneModel-TSS-Constant-1_cor
9,HLA-DRB1,0.8767254,-0.2033642,0.5332712,0.55744376,1.080090,GeneModel-GB-NoBoundary-Exponential-4_cor,GeneModel-TSS-Constant-4_cor


Genes that are best correlated to a particular model:

In [49]:
cor_df %>%
  arrange(desc(max_cor)) %>%
  head(20)

,gene,max_cor,min_cor,mean_cor,median_cor,diff_cor,max_model,min_model
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
1,WDFY4,0.9834847,0.93834273,0.9741367,0.9766379,0.04514200,GeneModel-TSS-NoBoundary-Exponential-2_cor,GeneModel-TSS-Constant-1_cor
2,LEF1,0.9801195,0.89852582,0.9710490,0.9733219,0.08159367,GeneModel-GB-Exponential-Extend-15_cor,GeneModel-TSS-Constant-1_cor
3,LYN,0.9788781,0.85975641,0.9739401,0.9784661,0.11912171,GeneModel-GB-Exponential-Extend-12_cor,GeneModel-TSS-Constant-1_cor
4,ARHGAP24,0.9750289,0.95100635,0.9648277,0.9637603,0.02402258,GeneModel-GB-Exponential-Extend-1_cor,GeneModel-TSS-Constant-2_cor
5,LEF1,0.9731400,0.92948049,0.9551207,0.9482880,0.04365948,GeneBody_0_0_cor,GeneModel-TSS-Exponential-1_cor
6,BLK,0.9706990,0.92649740,0.9456374,0.9511926,0.04420161,GeneModel-TSS-Exponential-2_cor,GeneBody_0_0_cor
7,HLA-DQA1,0.9685488,0.89994738,0.9357027,0.9400407,0.06860137,GeneModel-TSS-Constant-2_cor,GeneModel-TSS-Constant-3_cor
8,MS4A1,0.9649947,0.84082852,0.9157846,0.9572178,0.12416619,GeneModel-GB-Exponential-Extend-17_cor,GeneBody_0_0_cor
9,SYK,0.9632573,0.90587080,0.9582724,0.9599529,0.05738652,GeneModel-GB-Exponential-2_cor,GeneModel-TSS-Constant-1_cor


Genes that can be correlated, but are pretty much model-agnostic:

In [52]:
cor_df %>%
  filter(max_cor > 0.3) %>%
  arrange(diff_cor) %>%
  head(20)

,gene,max_cor,min_cor,mean_cor,median_cor,diff_cor,max_model,min_model
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
1,HOOK1,0.6580174,0.6500718,0.6539771,0.6550967,0.007945667,GeneModel-GB-Exponential-1_cor,GeneModel-TSS-Constant-4_cor
2,LINC00944,0.8232432,0.8151246,0.8196423,0.8200386,0.008118615,GeneModel-GB-Exponential-Extend-18_cor,GeneModel-TSS-Constant-1_cor
3,RAB32,0.8941697,0.8854335,0.8903455,0.8910311,0.008736149,GeneModel-TSS-NoBoundary-Exponential-1_cor,GeneModel-TSS-Constant-3_cor
4,PAWR,0.3247299,0.3148046,0.3210925,0.3208496,0.009925276,GeneModel-TSS-NoBoundary-Exponential-2_cor,GeneModel-TSS-Constant-4_cor
5,BEX5,0.3234097,0.3116577,0.3175291,0.3190257,0.011752088,GeneModel-TSS-Constant-1_cor,GeneModel-GB-NoBoundary-Exponential-4_cor
6,SPART,0.4557216,0.4430177,0.4488895,0.4480560,0.012703922,GeneModel-TSS-Exponential-1_cor,GeneModel-GB-NoBoundary-Exponential-4_cor
7,HLA-DMA,0.9230431,0.9090159,0.9177727,0.9168556,0.014027206,GeneModel-GB-NoBoundary-Exponential-1_cor,GeneModel-TSS-Constant-3_cor
8,HLA-DPA1,0.9108534,0.8966471,0.9054856,0.9054880,0.014206274,GeneModel-GB-NoBoundary-Exponential-1_cor,GeneModel-TSS-Constant-1_cor
9,CLEC11A,0.5413528,0.5267705,0.5332624,0.5333317,0.014582318,GeneModel-GB-NoBoundary-Exponential-2_cor,GeneModel-TSS-Constant-1_cor


In [40]:
max_model_tbl <- table(cor_df$max_model)
max_model_tbl <- max_model_tbl[order(max_model_tbl, decreasing = TRUE)]
max_model_tbl


                          GeneBody_0_0_cor 
                                      5214 
 GeneModel-GB-NoBoundary-Exponential-4_cor 
                                      2969 
              GeneModel-TSS-Constant-1_cor 
                                      2457 
              GeneModel-TSS-Constant-4_cor 
                                      2179 
              GeneModel-TSS-Constant-3_cor 
                                      1552 
              GeneModel-TSS-Constant-2_cor 
                                      1267 
            GeneModel-GB-Exponential-4_cor 
                                      1253 
    GeneModel-GB-Exponential-Extend-18_cor 
                                       833 
 GeneModel-GB-NoBoundary-Exponential-1_cor 
                                       791 
    GeneModel-GB-Exponential-Extend-12_cor 
                                       727 
GeneModel-TSS-NoBoundary-Exponential-1_cor 
                                       694 
 GeneModel-GB-NoBoundary-Expone

In [41]:
min_model_tbl <- table(cor_df$min_model)
min_model_tbl <- min_model_tbl[order(min_model_tbl, decreasing = TRUE)]
min_model_tbl


              GeneModel-TSS-Constant-1_cor 
                                      5259 
                          GeneBody_0_0_cor 
                                      5175 
 GeneModel-GB-NoBoundary-Exponential-4_cor 
                                      3123 
              GeneModel-TSS-Constant-2_cor 
                                      2297 
              GeneModel-TSS-Constant-4_cor 
                                      2257 
              GeneModel-TSS-Constant-3_cor 
                                      1921 
            GeneModel-GB-Exponential-4_cor 
                                       889 
GeneModel-TSS-NoBoundary-Exponential-1_cor 
                                       458 
    GeneModel-GB-Exponential-Extend-12_cor 
                                       457 
    GeneModel-GB-Exponential-Extend-18_cor 
                                       443 
            GeneModel-GB-Exponential-1_cor 
                                       422 
    GeneModel-GB-Exponential-Ex

### For each model, what are the mean/median/quantiles

In [45]:
model_df <- data.frame(
    model_name = colnames(res_mat),
    model_mean = colMeans(res_mat),
    model_min = colMins(res_mat),
    model_q25 = apply(res_mat, 2, quantile, prob = 0.25),
    model_median = colMedians(res_mat),
    model_q75 = apply(res_mat, 2, quantile, prob = 0.75),
    model_max = colMaxs(res_mat)
)

In [48]:
model_df

,model_name,model_mean,model_min,model_q25,model_median,model_q75,model_max
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GeneBody_0_0_cor,GeneBody_0_0_cor,0.11992961,-0.6674986,0.000000000,0.04250600,0.1782123,0.9805843
GeneBody_1000_0_cor,GeneBody_1000_0_cor,0.11992961,-0.6674986,0.000000000,0.04250600,0.1782123,0.9805843
GeneBody_1000_1000_cor,GeneBody_1000_1000_cor,0.11992961,-0.6674986,0.000000000,0.04250600,0.1782123,0.9805843
GeneBody_10000_0_cor,GeneBody_10000_0_cor,0.11992961,-0.6674986,0.000000000,0.04250600,0.1782123,0.9805843
GeneBody_10000_10000_cor,GeneBody_10000_10000_cor,0.11992961,-0.6674986,0.000000000,0.04250600,0.1782123,0.9805843
GeneBody_2000_0_cor,GeneBody_2000_0_cor,0.11992961,-0.6674986,0.000000000,0.04250600,0.1782123,0.9805843
GeneBody_2000_2000_cor,GeneBody_2000_2000_cor,0.11992961,-0.6674986,0.000000000,0.04250600,0.1782123,0.9805843
GeneBody_5000_0_cor,GeneBody_5000_0_cor,0.11992961,-0.6674986,0.000000000,0.04250600,0.1782123,0.9805843
GeneBody_5000_5000_cor,GeneBody_5000_5000_cor,0.11992961,-0.6674986,0.000000000,0.04250600,0.1782123,0.9805843
